# LSTM classification model for automatically mark the behavior states of mice.
The goal is to mark the state of animals of each frame.

*   sniffing
*   digging
*   eating
*   mobile
*   immobile


Steps: 

1. define features and its calculation.
2. creat trainning, validation, test dataset.
3. creat model.
4. feed data to model. 
5. evaluate the model.
6. predict new data by using the model. 
7. generate the "state" as new column in the .csv file.
8. show the text with video to check the correction ratio.





Define features

* nose_speed
* nose_acc
* head_centre_speed
* head_centre_acc
* angle_head_centre_nose (with vertical)
* angle_head_centre_mid_ears (with vertical）
* area_head (nose, left_ear, right_ear)
* dist_nose_dish_y (y_nose - mean(objs_y))
* body_area_8lines(upper_back, upper_left_side, mid_left_side, left_hip, mid_hip, right_hip, mid_right_side, upper_right_side)
* angle_left(upper_left_side-mid_left_side, mid_left_side-left_hip, clockwise/counterclockwise)
* angle_right(upper_right_side - mid_right_side, mid_right_side - right_hip, clockwise/counterclockwise)
* angle_mid(upper_back, mid_back, mid_hip)
* angle_consistent（angle_left, angle_right, plus/minus）


I do not want to add tails related values as features. Here are the reasons:
1. tails, especially distal tail, is not marked well
2. tails is also depend on the position the mice stand on, because he need to keep balance. but this is an unrelavant part. 
3. the features I pick up are the most important part.



In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

import os
os.getcwd()

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

Mounted at /content/gdrive


When I start to extract features, I should considering about these factor
1. select the datapoint from DLC that has likelihood > 0.95
2. has some variation, which means the model can learn something from the variation. The features like obj_L is not variated, so should not be a feature.
3. only keep one feature in the features that have highly correlation. Keep the one that has a higher coefficient with the target.


In [2]:
# add markers according to the video.
df = pd.read_csv('/content/gdrive/MyDrive/DataColab/2021_12_30-19_51_18 GL13533 RLT1st APCR trial3DLC_resnet_50_RLT-calFeb3shuffle1_310000.csv')
df.head()


,scorer,DLC_resnet_50_RLT-calFeb3shuffle1_310000,DLC_resnet_50_RLT-calFeb3shuffle1_310000.1,DLC_resnet_50_RLT-calFeb3shuffle1_310000.2,DLC_resnet_50_RLT-calFeb3shuffle1_310000.3,DLC_resnet_50_RLT-calFeb3shuffle1_310000.4,DLC_resnet_50_RLT-calFeb3shuffle1_310000.5,DLC_resnet_50_RLT-calFeb3shuffle1_310000.6,DLC_resnet_50_RLT-calFeb3shuffle1_310000.7,DLC_resnet_50_RLT-calFeb3shuffle1_310000.8,...,DLC_resnet_50_RLT-calFeb3shuffle1_310000.50,DLC_resnet_50_RLT-calFeb3shuffle1_310000.51,DLC_resnet_50_RLT-calFeb3shuffle1_310000.52,DLC_resnet_50_RLT-calFeb3shuffle1_310000.53,DLC_resnet_50_RLT-calFeb3shuffle1_310000.54,DLC_resnet_50_RLT-calFeb3shuffle1_310000.55,DLC_resnet_50_RLT-calFeb3shuffle1_310000.56,DLC_resnet_50_RLT-calFeb3shuffle1_310000.57,DLC_resnet_50_RLT-calFeb3shuffle1_310000.58,DLC_resnet_50_RLT-calFeb3shuffle1_310000.59
0,bodyparts,nose,nose,nose,head_centre,head_centre,head_centre,left_ear,left_ear,left_ear,...,obj_A_left,obj_A_right,obj_A_right,obj_A_right,obj_B_left,obj_B_left,obj_B_left,obj_B_right,obj_B_right,obj_B_right
1,coords,x,y,likelihood,x,y,likelihood,x,y,likelihood,...,likelihood,x,y,likelihood,x,y,likelihood,x,y,likelihood
2,0,1381.857177734375,854.02734375,0.8865680694580078,1528.15625,830.9244995117188,0.008904248476028442,1461.06689453125,819.2879028320312,0.025114715099334717,...,0.9759587645530701,1093.4736328125,642.5014038085938,0.9862077236175537,604.3071899414062,647.7957153320312,0.7539091110229492,829.9374389648438,648.9163208007812,0.9443341493606567
3,1,1381.852294921875,853.9972534179688,0.8887299299240112,1528.1744384765625,830.9308471679688,0.00907832384109497,1460.9405517578125,819.3567504882812,0.026066750288009644,...,0.9759613275527954,1093.4720458984375,642.4988403320312,0.9861567616462708,604.3070068359375,647.7944946289062,0.7536468505859375,829.9365234375,648.9154663085938,0.944310188293457
4,2,1381.56591796875,854.2176513671875,0.8892529606819153,1456.072998046875,842.8145751953125,0.0195942223072052,1464.04052734375,838.9147338867188,0.04577738046646118,...,0.9763677716255188,1093.448486328125,642.4821166992188,0.9864065647125244,604.5034790039062,647.7381591796875,0.7415812015533447,829.9202270507812,648.9358520507812,0.944926917552948
